<a href="https://colab.research.google.com/github/LiamTodd/inclusiviser-mvp/blob/main/Alpaca_LLM_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**FastAPI for alpaca-lora-7b LLM**

This is for development and testing purposes only and is not fit for production.

Set the runtime to GPU.

To access the API endpoint, run the FastAPI server and use the ngrok temporary public url as the base.

# Alpaca model
sources:
- https://colab.research.google.com/drive/1FUIag7sSnZ94nnoWAMJqY2lSORDq75sW?usp=sharing#scrollTo=ph4BKMCFgXms
- https://www.youtube.com/watch?v=JzBR8oieyy8


In [1]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00
     ━━

In [3]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [88]:
def alpaca_talk(text, temperature=0.5, top_p=0.95, repetition_penalty=1.2, max_new_tokens=512):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
    )
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=max_new_tokens,
    )
    for s in generation_output.sequences:
        return tokenizer.decode(s)

#Prompt engineering
Without a prompt template, the LLM simply replies with the words with the highest probability of being next. Thus, a prompt like "Hello, I am the user" may yield an output such as "Hello, I am the user of a laptop on a regular basis.".

As such, a template is needed in order to guide the LLM to behave as an instruction-following LLM.

In [54]:
def get_basic_prompt(instruction):
  return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Instruction:{instruction}
  ### Response:
  """

##Demonstration of the model outputs with/without prompt template

In [59]:
print("Generated without template:")
prompt="Reply with 'hello from alpaca'"
reply = alpaca_talk(prompt)
print(reply)
print("------------------")

print("Generated with template:")
prompt = get_basic_prompt("Reply with 'hello from alpaca'")
reply = alpaca_talk(prompt)
print(reply)

Generated without template:
 Reply with 'hello from alpaca'
I am a 10 year old girl who loves to read and write. I have been writing stories since i was about five years old, but now that im older it has become more of an obsession than just something fun to do! My favourite genres are fantasy/sci-fi (i love anything magical) and mystery/thriller.
------------------
Generated with template:
 Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Instruction:Reply with 'hello from alpaca'
  ### Response:
   Hello from Alpaca!


## Prompt testing
Determining prompts capable of suggesting changes to be made to files including non-inclusive language.

In [149]:
def get_inclusifier_prompt_non_code(input, terms):
  return f"""
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  INSTRUCTION: Rewrite the text without these terms: {", ".join(terms)}. Provide only one response.
  TEXT: {input}
  RESPONSE:
  """

def get_inclusifier_prompt_non_code_retain(input, terms):
    return f"""
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  INSTRUCTION: Rewrite the text without these terms: {", ".join(terms)}. Retain the full original meaning and all details. Provide only one response.
  TEXT: {input}
  RESPONSE:
  """

def get_inclusifier_prompt_non_code_inclusive(input, terms):
  return f"""
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  INSTRUCTION: Make the following text more inclusive by avoiding the use of these words: {", ".join(terms)}.
  TEXT: {input}
  RESPONSE:
  """

def get_inclusifier_prompt_non_code_category(input, categories):
  return f"""
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  INSTRUCTION: Rewrite the following text so that it could not be considered {", ".join(categories)}. Retain the full original meaning and all details. Provide only one response.
  TEXT: {input}
  RESPONSE:
  """


In [157]:
input = """
Once the first child process returns, the parent will kill all the rest of the children.
"""
# terms = ["jank"]
categories = ["violent"]
prompt1 = get_inclusifier_prompt_non_code_category(input, categories)
# prompt2 = get_inclusifier_prompt_non_code_inclusive(input, terms)
print(alpaca_talk(prompt1))
print("--------------------")
# print(alpaca_talk(prompt2))

 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  INSTRUCTION: Rewrite the following text so that it could not be considered violent. Retain the full original meaning and all details. Provide only one response.
  TEXT: 
Once the first child process returns, the parent will kill all the rest of the children.

  RESPONSE:
   Once the first child process has completed its work, the parent will terminate any remaining processes.
--------------------


# FastAPI
source: https://medium.datadriveninvestor.com/flask-on-colab-825d2099d9d8

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install fastapi nest-asyncio pyngrok uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=1f4f637a540662085dda49ae856e44200f88e531dca989b7257fb112df987d4b
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/index')
async def home():
  return alpaca_talk("""
  SYSTEM: you are an honest and helpful assistant, reply as ASSISTANT.
  USER: Reply with "Hello from alpaca" and NOTHING ELSE
  ASSISTANT:
  """)

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Exception in thread Thread-14 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 146, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


Public URL: https://a7f0-34-124-205-132.ngrok.io


INFO:     Started server process [352]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2001:8003:401a:c800:8d5e:d058:b40d:b0bf:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2001:8003:401a:c800:8d5e:d058:b40d:b0bf:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
Generating...
INFO:     2001:8003:401a:c800:8d5e:d058:b40d:b0bf:0 - "GET /index HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [352]
